In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

#Función que te permite obtener la fila i-ésima de la columna 'hist'
#Se introduce el dataframe y el número de fila
def str_array(df, i):
    return [np.array(eval(df.values[i])).astype(int)]


# Función para generar la matriz con las componentes principales tras el entrenamiento
#Se introduce el diccionario con las propiedades separadas, el diccionario de matrices V de componentes principales, la propiedad que se entrena y el número de componentes
# El quinto parámetro permite obtener información sobre la varianza y el número de componentes principales utilizadas si se hace igual a 1
def training(dic_prop_era, dic_V, prop, era, PC, exp):
    matriz = np.zeros((1,np.shape(dic_V['V{}A'.format(prop)])[1]))  #Se crea una fila de ceros para concatenar sobre ella, luego se elimina
    for i in era:
        ind_buenos = np.array(dic_prop_era['{}{}'.format(prop,i)][dic_prop_era['{}{}'.format(prop,i)]['labels']==True]['new_lumi'].astype(int))
        datos = dic_V['V{}{}'.format(prop, i)][ind_buenos]
        matriz = np.concatenate((matriz,datos), axis=0)
    matriz = np.delete(matriz,0,axis=0)
    pca = PCA(n_components = PC)
    pca.fit(matriz)
    if exp==1:
        print('Varianza explicada por componente:', pca.explained_variance_ratio_*100)
        print('Varianza explicada total:', np.sum(pca.explained_variance_ratio_*100))
        print('Número de componentes principales:', pca.n_components_)
    return pca.components_


#Función que permite evaluar el modelo. Se introduce la matriz V de los datos a evaluar y las componentes tras el entrenamiento
def evaluation(datos, componentes):
    recon = np.matmul(np.matmul(datos, componentes.T), componentes) #Cada fila es un histograma reconstruido normalizado
    norma = np.max([np.max(datos, axis = 1), np.max(recon, axis = 1)], axis = 0) #Para calcular el MSE se normalizan los histogramas
    mse = (((datos-recon)/norma[:, None])**2).sum(axis=1)/np.shape(datos)[1]
    return (recon, mse)


from scipy.signal import find_peaks
#Función que determina las regiones anómalas
#Se introducen los datos suavizados y la anchura mínima que deben tener los picos para ser señalados como datos malos
def anomalias(datos_suavizados, anchura):
    maximos,_ = find_peaks(datos_suavizados, width=150, distance=50) #Se determinan los máximos asociados a fills
    picos, picosinfo = find_peaks(datos_suavizados, width=anchura, distance=50, prominence=np.min(datos_suavizados)/2) #Picos anómalos en los datos suavizados
    semianchuras = np.asarray(np.floor(picosinfo['widths']/2), dtype='int')
    descartes = np.array([picos, semianchuras])  #Matriz. Primera fila son los picos, la segunda sus semianchuras como números enteros
    descartes = descartes[:, ~np.isin(descartes[0],maximos)]  #Quitamos los picos que sean máximos debidos a fills
    descartes = descartes[:, descartes[0]<(len(datos_suavizados)-150)]
    descartes = descartes[:, descartes[0]>200]
    extra = [0]
    for n in range(200,len(datos_suavizados)-200):
        if datos_suavizados[n]>2*np.median(datos_suavizados[(n-10):n]) and extra[len(extra)-1]<(n-100):
            extra = np.append(extra,n)
            k = 1
            while datos_suavizados[n+k] < 1.5*datos_suavizados[n+k+50]:
                k = k+1
            descartes = np.c_[descartes, [n+np.floor(k/2), np.floor(k/2)]]
    descartes = descartes.astype(int)
    return descartes, maximos

#Función análoga a la interior pero adaptada a las particularidades de chi2OverDf
def anomaliaschi(datos_suavizados, anchura):
    picos, picosinfo = find_peaks(datos_suavizados, width=anchura, distance=10, prominence=np.min(datos_suavizados))
    semianchuras = np.asarray(np.floor(picosinfo['widths']/2), dtype='int')
    descartes = np.array([picos, semianchuras])  #Matriz. Primera fila son los picos, la segunda sus semianchuras como números enteros
    descartes = descartes[:, descartes[0]<(len(datos_suavizados)-150)]
    descartes = descartes[:, descartes[0]>200]
    return descartes


#Función para determinar el parámetro de corte y los datos descartados a partir de él
#Se introduce el vector de mse, los datos suavizados y los máximos debidos a fills calculados con la función anomalias
def corte(datos, datos_suavizados, maximos):
    id = np.max(np.where(datos_suavizados[0]==np.max(datos_suavizados[0][maximos]))) #Se determina el índice en el que se alcanza el máximo de la mediana móvil
    vmax = 0
    #Se escanea la zona alrededor de dicho máximo
    for n in range(id-200,id+200):
        if datos[n]>vmax and datos[n]<2*np.median(datos[(n-100):n]):
            vmax = datos[n]
            id2 = n
    falsos_corte = np.where(datos>vmax)[0] #Índices de los valores descartados por superar el parámetro de corte
    return vmax, falsos_corte

#Análoga de la función anterior adaptada a las particularidades de chi2OverDf
#Se introduce únicamente el mse
def cortechi(datos):
    id = np.where(datos>np.percentile(datos,99))[0]
    for n in id:
        if(datos[n]<10*np.median(datos[(n-min(n,100)):(n+min(len(datos)-n, 100))])):
            id = id[id!=n]
    if len(id)>0:
        vmax = min(datos[id])
    else:
        vmax = 10**10
    falsos_corte = np.where(datos>vmax)[0]
    return vmax, falsos_corte

#Permite obtener los datos etiquetados como malos por el modelo completo
#Se introduce el diccionario de conjuntos, el diccionario de matrices V, una lista con las eras de entrenamiento, una con las de evaluación y una con el número de PC
def datos_malos(dic_prop_era,dic_V,entrenamiento,evaluacion,n):
    malos = np.array([0])
    k = 0
    for i in ['eta','chi2OverDf','phi','pt']:
        matriz = np.zeros((1,np.shape(dic_V['V{}A'.format(i)])[1]))
        for t in evaluacion:
            datos_eval = dic_V['V{}{}'.format(i, t)]
            matriz = np.concatenate((matriz,datos_eval), axis=0)
        matriz = np.delete(matriz,0,axis=0)
        if type(n)==int:
            recon, mse = evaluation(matriz, training(dic_prop_era, dic_V, i, entrenamiento, n, 0))
        else:
            recon, mse = evaluation(matriz, training(dic_prop_era, dic_V, i, entrenamiento, n[k], 0))
            k = k+1
        mse_df = pd.DataFrame(mse)
        smoothed_mse_median = mse_df.rolling(window = 100, center=True).apply(np.median)
        if i=='chi2OverDf':
            descartes = anomaliaschi(smoothed_mse_median[0], 10**10)
            vmax, falsos_corte = cortechi(mse)
        else:
            descartes, maximos = anomalias(smoothed_mse_median[0], 30)
            vmax, falsos_corte = corte(mse, smoothed_mse_median, maximos)
        falsos = falsos_corte
        for j in range(0,len(descartes[0])):
            ei = descartes[0,j]-descartes[1,j]
            ed = descartes[0,j]+descartes[1,j]
            e = 2*descartes[1,j]+1
            falsos = np.concatenate((falsos,np.linspace(ei,ed,e)))
        malos = np.concatenate((malos,falsos))
    malos = np.unique(malos[malos!=0]).astype(int)
    return malos, mse

#Análoga de la función anterior pero usa una única propiedad como criterio de descarte
#Se añade el parámetro que indica la propiedad usada como un string
def datos_malos_PC(dic_prop_era,dic_V,entrenamiento,evaluacion,n,prop):
    malos = np.array([0])
    k = 0
    for i in ['{}'.format(prop)]:
        matriz = np.zeros((1,np.shape(dic_V['V{}A'.format(i)])[1]))
        for t in evaluacion:
            datos_eval = dic_V['V{}{}'.format(i, t)]
            matriz = np.concatenate((matriz,datos_eval), axis=0)
        matriz = np.delete(matriz,0,axis=0)
        if type(n)==int:
            recon, mse = evaluation(matriz, training(dic_prop_era, dic_V, i, entrenamiento, n, 0))
        else:
            recon, mse = evaluation(matriz, training(dic_prop_era, dic_V, i, entrenamiento, n[k], 0))
            k = k+1
        mse_df = pd.DataFrame(mse)
        smoothed_mse_median = mse_df.rolling(window = 100, center=True).apply(np.median)
        if i=='chi2OverDf':
            descartes = anomaliaschi(smoothed_mse_median[0], 10**10)
            vmax, falsos_corte = cortechi(mse)
        else:
            descartes, maximos = anomalias(smoothed_mse_median[0], 30)
            vmax, falsos_corte = corte(mse, smoothed_mse_median, maximos)
        falsos = falsos_corte
        for j in range(0,len(descartes[0])):
            ei = descartes[0,j]-descartes[1,j]
            ed = descartes[0,j]+descartes[1,j]
            e = 2*descartes[1,j]+1
            falsos = np.concatenate((falsos,np.linspace(ei,ed,e)))
        malos = np.concatenate((malos,falsos))
    malos = np.unique(malos[malos!=0]).astype(int)
    return malos, mse

#Obtiene la matriz de confusión y las medidas resumen
#Se introduce el diccionario de conjuntos, la lista de eras de evaluación y los datos malos obtenidos de la función datos_malos
#El último parámetro muestra por pantalla los parámetros si es igual a 1
def resultados(dic_prop_era,evaluacion,malos,exp):
    df = dic_prop_era['eta{}'.format(evaluacion[0])]
    etiquetas_expertos = np.array(df['labels'])
    etiquetas_modelo = ~np.isin(np.linspace(0,len(etiquetas_expertos),len(etiquetas_expertos)).astype(int),malos)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(0,len(etiquetas_expertos)):
        if etiquetas_expertos[i]==True and etiquetas_modelo[i]==True:
            TP = TP+1
        elif etiquetas_expertos[i]==False and etiquetas_modelo[i]==False:
            TN = TN+1
        elif etiquetas_expertos[i]==True and etiquetas_modelo[i]==False:
            FN = FN+1
        else:
            FP = FP+1
    PPV = TP/(TP+FP)
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)
    F12 = (1+0.25)*PPV*TPR/(0.25*PPV+TPR)
    GM = np.sqrt(TPR*TNR)
    if exp==1:
        print('TP:',TP,'  FN:',FN)
        print('FP:',FP,'     TN:',TN)
        print('PPV:',PPV)
        print('TPR:',TPR)
        print('TNR:',TNR)
        print('F12:',F12)
        print('GM:',GM)
    return F12, GM

#Análogo de la función anterior para obtener únicamente la matriz de confusión
def resultados_prop(dic_prop_era,evaluacion,malos,exp):
    df = dic_prop_era['eta{}'.format(evaluacion[0])]
    etiquetas_expertos = np.array(df['labels'])
    etiquetas_modelo = ~np.isin(np.linspace(0,len(etiquetas_expertos),len(etiquetas_expertos)).astype(int),malos)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(0,len(etiquetas_expertos)):
        if etiquetas_expertos[i]==True and etiquetas_modelo[i]==True:
            TP = TP+1
        elif etiquetas_expertos[i]==False and etiquetas_modelo[i]==False:
            TN = TN+1
        elif etiquetas_expertos[i]==True and etiquetas_modelo[i]==False:
            FN = FN+1
        else:
            FP = FP+1
    PPV = TP/(TP+FP)
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)
    F12 = (1+0.25)*PPV*TPR/(0.25*PPV+TPR)
    GM = np.sqrt(TPR*TNR)
    if exp==1:
        print('TP:',TP,'  FN:',FN)
        print('FP:',FP,'     TN:',TN)
        print('PPV:',PPV)
        print('TPR:',TPR)
        print('TNR:',TNR)
        print('F12:',F12)
        print('GM:',GM)
    return TP, FP, FN, TN

#Análoga de datos_malos pero usando la media en lugar de la mediana
def datos_malos_mean(dic_prop_era,dic_V,entrenamiento,evaluacion,n):
    malos = np.array([0])
    k = 0
    for i in ['eta','chi2OverDf','phi','pt']:
        matriz = np.zeros((1,np.shape(dic_V['V{}A'.format(i)])[1]))
        for t in evaluacion:
            datos_eval = dic_V['V{}{}'.format(i, t)]
            matriz = np.concatenate((matriz,datos_eval), axis=0)
        matriz = np.delete(matriz,0,axis=0)
        if type(n)==int:
            recon, mse = evaluation(matriz, training(dic_prop_era, dic_V, i, entrenamiento, n, 0))
        else:
            recon, mse = evaluation(matriz, training(dic_prop_era, dic_V, i, entrenamiento, n[k], 0))
            k = k+1
        mse_df = pd.DataFrame(mse)
        smoothed_mse_mean = mse_df.rolling(window = 100, center=True).mean()
        if i=='chi2OverDf':
            descartes = anomaliaschi(smoothed_mse_mean[0], 10**10)
            vmax, falsos_corte = cortechi(mse)
        else:
            descartes, maximos = anomalias(smoothed_mse_mean[0], 30)
            vmax, falsos_corte = corte(mse, smoothed_mse_mean, maximos)
        falsos = falsos_corte
        for j in range(0,len(descartes[0])):
            ei = descartes[0,j]-descartes[1,j]
            ed = descartes[0,j]+descartes[1,j]
            e = 2*descartes[1,j]+1
            falsos = np.concatenate((falsos,np.linspace(ei,ed,e)))
        malos = np.concatenate((malos,falsos))
    malos = np.unique(malos[malos!=0]).astype(int)
    return malos, mse